In [3]:
from numpy import savez, save, load, array
import os
from keras.layers import Dense, Flatten, Dropout
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from numpy.random import seed, shuffle
import h5py
from numpy import asarray

/Users/sunny/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
def make_dataset(path):
    dataset = []
    num_categories = 38
    labels = []
    for i in range(38):
        cat_path = path+'/c_'+str(i)
        for img in os.listdir(cat_path):
            temp = image.load_img(cat_path +'/'+img, target_size=(224,224,3))
            temp = image.img_to_array(temp)
            dataset.append(temp)
            labels.append(i)
        print('Category ' + str(i)+' out of 37 ', end='\r')
    print('Processing the input and saving. This may take some time.')
    dataset = array(dataset)
    dataset = preprocess_input(dataset)
    labels = array(labels).reshape(-1,1)
    labels = to_categorical(labels, 38)
    with h5py.File('crop_disease_dataset.h5', 'w') as h5:
        h5.create_dataset('dataset', data=dataset)
        h5.create_dataset('labels', data=labels)
    print('Saved Dataset as .npz')
    return dataset, labels

In [5]:
train_path = 'crowdai'
train_dataset, train_labels = make_dataset(train_path)

Processing the input and saving. This may take some time in Vinay's Lappy.
Saved Dataset as .npz


In [2]:
with h5py.File('crop_disease_dataset.h5', 'r') as h5:
    train_dataset = h5.get('dataset')[:]
    train_labels = h5.get('labels')[:]
#print(train_dataset, train_labels)
train_dataset = array(train_dataset)
train_labels = array(train_labels)

In [11]:
#avoid this. this step is forward prop
base_model = VGG16(weights = 'imagenet', include_top = False)
train_features = base_model.predict(train_dataset, verbose=1)
save('train_features', train_features)

21917/21917 [==============================] - 7687s 351ms/step


In [4]:
train_features = train_features.reshape((21917,-1))

In [3]:
#avoid forward prop by doing this
train_features = load('train_features.npy')

In [5]:
seed(28)
from tensorflow import set_random_seed
set_random_seed(2)
c = list(zip(train_features, train_labels))
shuffle(c)
train_features, train_labels = zip(*c)
del c

In [6]:
train_features = asarray(train_features)
train_labels = asarray(train_labels)

In [7]:
model = Sequential()
#model.add(Flatten)
model.add(Dense(1024, input_dim=7*7*512, activation='relu', kernel_initializer='glorot_normal'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='sigmoid', kernel_initializer='glorot_normal'))
model.add(Dropout(0.5))
model.add(Dense(38, activation='softmax', kernel_initializer='glorot_normal'))

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(train_features, train_labels, batch_size = 128, epochs = 15, verbose=1, validation_split=0.1)

Train on 19725 samples, validate on 2192 samples
Epoch 1/15
19725/19725 [==============================] - 45s 2ms/step - loss: 2.2037 - acc: 0.4524 - val_loss: 0.9816 - val_acc: 0.7701
Epoch 2/15
19725/19725 [==============================] - 46s 2ms/step - loss: 1.1211 - acc: 0.7272 - val_loss: 0.5728 - val_acc: 0.8618
Epoch 3/15
19725/19725 [==============================] - 45s 2ms/step - loss: 0.7484 - acc: 0.8214 - val_loss: 0.4318 - val_acc: 0.8855
Epoch 4/15
19725/19725 [==============================] - 46s 2ms/step - loss: 0.5650 - acc: 0.8653 - val_loss: 0.3362 - val_acc: 0.8978
Epoch 5/15
19725/19725 [==============================] - 46s 2ms/step - loss: 0.4424 - acc: 0.8978 - val_loss: 0.2540 - val_acc: 0.9261
Epoch 6/15
19725/19725 [==============================] - 47s 2ms/step - loss: 0.3574 - acc: 0.9162 - val_loss: 0.2272 - val_acc: 0.9307
Epoch 7/15
19725/19725 [==============================] - 47s 2ms/step - loss: 0.3043 - acc: 0.9296 - val_loss: 0.2003 - val_acc:

In [29]:
model.save_weights('saved-weights_crop-disease.h5')
model.save('saved_model_crop_diseases.h5')

In [9]:
base_model = VGG16(weights = 'imagenet',include_top=False, input_shape=(224,224,3))
dense_model = load_model('saved_model_crop_diseases.h5')
base_model.outputs = [base_model.layers[-1].output]
base_model.layers[-1].outbound_nodes = []
bridge = base_model.layers[-1].output
x = Flatten()(bridge)
x = dense_model(x)
final_model = Model(inputs = base_model.input, outputs=x)

In [8]:
from keras.models import load_model
from keras import Model

In [15]:
final_model.save('complete_model_crop_diseases.h5')

In [1]:
index_map = {
    0:'Apple Scab',
    1:'Black Rot, Apple' ,
    2:'Cedar Rust, Apple',
    3:'Healthy Apple',
    4:'Healthy Blueberry',
    5: 'Powdery Mildew, Cherry',
    6:'Healthy Cherry',
    7:'Grey Leaf Spot, Corn',
    8: 'Common Rust of Maize',
    9:'Northern Leaf Blight, Corn',
    10:'Healthy Corn',
    11:'Black Rot, Grape' ,
    12: 'Black Measles, Grape',
    13:'Leaf Spot, Grape',
    14: 'Healthy Grape',
    15:'Citrus Huanglongbing',
    16: 'Bacterial Spot, Peach',
    17:'Healthy Peach',
    18:'Bacterial Spot, Bell Pepper',
    19:'Healthy Bell Pepper',
    20:'Early Blight, Potato',
    21:'Late Blight, Potato',
    22:'Healthy Potato',
    23:'Healthy Raspberry',
    24:'Healthy Soybean',
    25:'Powdery Mildew, Squash',
    26:'Leaf Scorch, Strawberry',
    27:'Healthy Strawberry',
    28:'Bacterial Leaf Spot, Tomato',
    29:'Early Blight, Tomato',
    30:'Late Blight, Tomato',
    31:'Leaf Mold, Tomato',
    32:'Leaf Spot, Tomato',
    33:'Two Spot Spider Mite, Tomato',
    34:'Target Leaf Spot, Tomato',
    35:'Yellow Leaf Curl, Tomato',
    36:'Mosaic, Tomato',
    37:'Healthy Tomato'
}

In [ ]:
temp = image.load_img('feabce4c-9bb1-4fca-bcbf-368cacd40a68___PSU_CG 2115.JPG', target_size=(224,224,3))
temp = image.img_to_array(temp)
temp = preprocess_input(temp)
temp = temp.reshape(1,224,224,3)
temp = final_model.predict(temp)
print(index_map[np.argmax(temp)])